<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN09/9.3-RNN_verstehen_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

This notebook contains the first code sample found in Chapter 6, Section 2 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

# 9.3 Rekurrente neuronale Netze verstehen

- Alle bislang vorgestellten Arten von NNs, wie z.B. Fully-connected CNNs, besitzen **kein »Gedächtnis«**. 
- Die Eingaben werden unabhängig voneinander verarbeitet – der durch Eingaben verursachte Zustand wird nicht gespeichert. 
- Um eine Sequenz oder eine Zeitreihe von Daten mit so einem NN verarbeiten zu können, muss die gesamte Sequenz auf einmal übergeben werden: Sie müssen sie also in einen einzigen Datenpunkt umwandeln. 

So sind wir beim IMDb-Beispiel vorgegangen: Eine komplette Filmbewertung wurde in einen einzelnen grossen Vektor umgewandelt
und »in einem Rutsch« verarbeitet. Man bezeichnet das als **Feedforward-Netz**.

- Im Gegensatz dazu verarbeiten Sie beim Lesen dieses Satzes die Wörter **der Reihe nach (sequentiell)** einzeln, nämlich bei jeder Augenbewegung, mit der Sie ein Wort erfassen. Dabei merken Sie sich die vorangegangenen Wörter und erhalten eine flüssige Repräsentation der Bedeutung des Satzes. 
- Biologische Intelligenz verarbeitet Informationen schrittweise und erzeugt dabei ein internes Modell dessen, was verarbeitet wird, das auf den erhaltenen Informationen aufbaut und kontinuierlich durch neue Informationen ergänzt wird.


<img src="Bilder/RNN_Schleife.PNG" width="500"  align="center"/>

Ein **rekurrentes neuronales Netz (RNN)** funktioniert nach dem gleichen Prinzip, wenngleich in einer extrem vereinfachten Version: Es verarbeitet Sequenzen, indem es die Elemente durchläuft und einen Zustand erfasst, der Informationen relativ zu
den vorangegangenen Elementen beinhaltet. 
- Faktisch ist ein RNN ein Netz, das eine **interne Schleife** besitzt. 
- Der Zustand des RNNs wird zwischen der Verarbeitung zweier unterschiedlicher, voneinander unabhängiger Sequenzen (etwa zwei verschiedene Filmbewertungen) zurückgesetzt, sodass Sie jede Sequenz nach wie vor als einzelnen Datenpunkt betrachten können, also als einzelne Eingabe für das RNN. 
- Der Unterschied besteht darin, dass dieser Datenpunkt nicht mehr in einem einzigen Schritt verarbeitet wird, sondern dass das RNN intern die Elemente der Sequenz in einer Schleife durchläuft.

### Test RNN

- Zur Verdeutlichung der Funktionsweise von Schleife und Zustand implementieren wir die Weitergabe des Zustands in einem *Test-RNN mit Numpy*.
- Dieses RNN nimmt eine Vektorsequenz als Eingabe entgegen, die als 2-D-Tensor der Grösse `(timesteps, input_features)` codiert ist. 
- Beim Durchlaufen der einzelnen Zeitschritte berücksichtigt es den aktuellen Zustand und die Eingabe mit der Shape `(input_features,)` zum Zeitpunkt `t` und berechnet daraus die Ausgabe für diesen Zeitpunkt `t`. 
- Anschliessend wird die Ausgabe dem Zustand für die nächste Iteration zugewiesen. Beim ersten Zeitschritt ist die vorangegangene Ausgabe undefiniert, und es gibt noch keinen aktuellen Zustand. 
- Er wird deshalb mit einem Nullvektor initialisiert, der als der Anfangszustand des RNNs bezeichnet wird.

Wir können die Funktion `f()` sogar schon konkretisieren: Die Abbildung der Eingabe
und des Zustands auf eine Ausgabe wird durch die beiden Matrizen `W` und `U`
sowie einen *Bias-Vektor* parametrisiert. Sie ähnelt der Abbildung, die ein Fullyconnected
Layer in einem Feedforward-Netz durchführt.

Zwecks Vermeidung von Mehrdeutigkeiten programmieren wir eine naive
Numpy-Implementierung für die Weitergabe des Zustands des einfachen RNNs.

In [ ]:
import numpy as np

# Anzahl der Zeitschritte in der Eingabesequenz
timesteps = 100
# Dimensionalität des Eingabemerkmalsraums
input_features = 32
# Dimensionalität des Ausgabemerkmalsraums
output_features = 64
# Eingabedaten, im Beispiel das zufällige Rauschen
inputs = np.random.random((timesteps, input_features))
# Anfangszustand, ein Nullvektor
state_t = np.zeros((output_features,))
# Erzeugt zufällige Gewichtungsmatrizen
W = np.random.random((output_features, input_features))
U = np.random.random((output_features, output_features))
b = np.random.random((output_features,))

successive_outputs = []
# input_t ist ein Vektor mit der Shape (input_features,).

for input_t in inputs:
    # Errechnet aus der Eingabe und dem aktuellen Zustand die Ausgabe
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    # Speichert die Ausgabe in einer Liste
    successive_outputs.append(output_t)
    # Aktualisiert den Zustand des Netzes für den nächsten Zeitschritt
    state_t = output_t
    final_output_sequence = np.concatenate(successive_outputs, axis=0)


**Zusammengefasst, ist ein RNN eine `for`-Schleife, die die beim vorangegangenen
Durchlauf der Schleife berechneten Grössen wiederverwendet, nicht mehr und nicht weniger.**

Natürlich gibt es viele verschiedene RNNs, auf die diese Definition
zutrifft – dieses Beispiel ist eine der einfachsten Möglichkeiten, ein RNN zu
beschreiben. RNNs sind durch die Funktion gekennzeichnet, die zum nächsten
Schritt führt. In diesem Fall sieht sie so aus:
<img src="Bilder/RNN.png" width="640"  align="center"/>


In diesem Beispiel ist die endgültige Ausgabe ein Tensor mit der Shape `(timesteps, output_features)`.
- Jeder Zeitschritt enthält die Ausgabe der Schleife zum Zeitpunkt `t`. 
- Der Zeitschritt `t` im Ausgabetensor enthält Informationen über die Zeitschritte `0` bis `t` in der Eingabesequenz – also über den gesamten zeitlichen Verlauf. 
- Aus diesem Grund ist in vielen Fällen die vollständige Ausgabesequenz gar nicht erforderlich. 
- Die letzte Ausgabe (`output_t` am Ende der Schleife) reicht aus, denn sie enthält Informationen über die gesamte Sequenz.

# Ein rekurrenter Layer in PyTorch

Ein einfacher rekurrenter Layer in PyTorch kann mit ``torch.nn.RNN`` umgesetzt werden. Dieser verarbeitet standardmäßig ganze Batches von Sequenzen.

- Die Eingabe muss die Form (seq_len, batch_size, input_size) haben. Alternativ kann man im Konstruktor batch_first=True setzen, dann wird die Form (batch_size, seq_len, input_size) erwartet.

- nn.RNN kann so verwendet werden, dass entweder:

    - die vollständige Ausgabesequenz für alle Zeitschritte zurückgegeben wird → Tensor der Form (batch_size, seq_len, hidden_size) (bei batch_first=True)

    - oder nur der letzte Hidden State verwendet wird → Tensor der Form (batch_size, hidden_size)

- Welche Variante man nimmt, hängt davon ab, ob man das komplette output-Tensor weiterverwendet oder nur das hidden-Tensor (hn).

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchinfo import summary


class SimpleRNN_LastOnly(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(10000, 32)
        self.rnn = nn.RNN(input_size=32, hidden_size=32, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)
        _, h = self.rnn(x)
        return h.squeeze(0)  # nur letzter Zustand


model1 = SimpleRNN_LastOnly().to("cpu")

summary(
    model1,
    input_size=(1, 100),  # (batch_size, sequence_length)
    dtypes=[torch.long],  # dtypes MUSS Liste sein!
    device="cpu",
)


Das folgende Beispiel gibt die vollständige Sequenz der Zustände zurück:

In [ ]:
class SimpleRNN_WithSequences(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(10000, 32)
        self.rnn = nn.RNN(input_size=32, hidden_size=32, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embedding_dim)
        output, _ = self.rnn(x)  # return_sequences=True → full output
        return output  # (batch, seq_len, hidden_size)


# Modellinstanz
model2 = SimpleRNN_WithSequences().to("cpu")

# torchinfo summary-Aufruf
summary(
    model2,
    input_size=(1, 100),  # (batch_size, sequence_length)
    dtypes=[torch.long],  # für nn.Embedding wichtig
    device="cpu",
)


Gelegentlich ist es sinnvoll, mehrere rekurrente Layer hintereinanderzuschalten,
um die Repräsentationsfähigkeit des NNs zu erhöhen. In diesem Fall müssen die
zwischenliegenden Layer die vollständigen Sequenzen ausgeben:

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchinfo import summary


class StackedSimpleRNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(10000, 32)
        self.rnn1 = nn.RNN(32, 32, batch_first=True)
        self.rnn2 = nn.RNN(32, 32, batch_first=True)
        self.rnn3 = nn.RNN(32, 32, batch_first=True)
        self.rnn4 = nn.RNN(32, 32, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embedding_dim)
        x, _ = self.rnn1(x)  # return_sequences=True → volle Sequenz
        x, _ = self.rnn2(x)  # "
        x, _ = self.rnn3(x)  # "
        _, h = self.rnn4(x)  # return_sequences=False → letzter Zustand
        return h.squeeze(0)  # (batch, hidden_size)


# Modell instanziieren
model = StackedSimpleRNN().to("cpu")

# torchinfo summary
summary(
    model,
    input_size=(1, 100),  # batch_size=1, sequence_length=100
    dtypes=[torch.long],  # nötig für Embedding
    device="cpu",
)


In [ ]:
import torch
from torchview import draw_graph

# Modell vorbereiten
model = StackedSimpleRNN().to("cpu")

# Visualisierung erstellen
viz = draw_graph(
    model,
    input_size=(1, 100),
    dtypes=[torch.long],
    device="cpu",
    expand_nested=True,
    graph_name="StackedSimpleRNN",
    save_graph=True,
    filename="model_plot",  # erzeugt model_plot.png
    directory="./",  # oder ein anderer Pfad
    show_shapes=True,
)


Nun verwenden wir ein solches Modell für die Aufgabe, IMDb-Filmbewertungen
zu klassifizieren. Zunächst erfolgt die Vorverarbeitung der Daten:

In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import re

# Hyperparameter
max_features = 10000  # Wortindex-Limit
maxlen = 500  # maximale Sequenzlänge
batch_size = 32


# Textvorverarbeitung: einfache Tokenisierung
def simple_tokenize(text):
    text = text.lower()
    text = re.sub(r"<br\s*/?>", " ", text)
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.split()


# 1. IMDb laden
print("Lade IMDb-Daten...")
dataset = load_dataset("imdb")
train_texts = dataset["train"]["text"]
train_labels = dataset["train"]["label"]
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

# 2. Tokenisierung & Vokabular-Erstellung
print("Tokenisiere Texte & baue Vokabular...")
tokenized_train = [simple_tokenize(text) for text in train_texts]
all_tokens = [token for text in tokenized_train for token in text]
most_common = Counter(all_tokens).most_common(max_features - 2)

# Spezial-Tokens
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
for idx, (word, _) in enumerate(most_common, start=2):
    word_to_idx[word] = idx


# Hilfsfunktion: Text zu Sequenz von Indizes
def encode(text_tokens):
    return [word_to_idx.get(token, word_to_idx["<UNK>"]) for token in text_tokens]


# 3. Kodieren und Padding
def encode_and_pad(text_list, maxlen):
    encoded = [encode(simple_tokenize(text)) for text in text_list]
    padded = []
    for seq in encoded:
        if len(seq) > maxlen:
            padded.append(seq[-maxlen:])  # abschneiden hinten
        else:
            padded.append([0] * (maxlen - len(seq)) + seq)  # vorne auffüllen mit PAD
    return torch.tensor(padded, dtype=torch.long)


print("Kodieren & Padding...")
X_train = encode_and_pad(train_texts, maxlen)
X_test = encode_and_pad(test_texts, maxlen)
y_train = torch.tensor(train_labels, dtype=torch.float32)
y_test = torch.tensor(test_labels, dtype=torch.float32)

# 4. Dataset & Dataloader
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

print(f"Train Shape: {X_train.shape}")
print(f"Test Shape: {X_test.shape}")


Und jetzt trainieren wir ein einfaches rekurrentes NN unter Verwendung eines
`Embedding`- und eines `SimpleRNN`-Layers.

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl


class SimpleRNNClassifier(pl.LightningModule):
    def __init__(self, vocab_size=10000, embedding_dim=32, hidden_size=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.loss_fn = nn.BCELoss()

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embedding_dim)
        _, h = self.rnn(x)  # h: (1, batch, hidden_size)
        out = self.fc(h.squeeze(0))  # (batch, 1)
        return torch.sigmoid(out)  # (batch, 1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)  # (batch,)
        loss = self.loss_fn(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        self.log("train_acc", acc, prog_bar=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = self.loss_fn(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True, on_epoch=True)
        self.log("val_acc", acc, prog_bar=True, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=1e-3)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
from pytorch_lightning.loggers import CSVLogger

np.Inf = np.inf
# TensorDataset + Validation-Split wie in validation_split=0.2
dataset = TensorDataset(X_train, y_train)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Lightning-Modell instanziieren
model = SimpleRNNClassifier(vocab_size=10000)


csv_logger = CSVLogger("logs", name="simple_rnn")
trainer = pl.Trainer(max_epochs=10, accelerator="auto", logger=csv_logger)
trainer.fit(model, train_loader, val_loader)


Zum Abschluss stellen wir die Verlustfunktion und die Korrektklassifizierungsrate
beim Training und bei der Validierung dar.

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt

# Load the metrics.csv file
metrics_file = "logs/simple_rnn/version_0/metrics.csv"  # Pfad anpassen!!
metrics = pd.read_csv(metrics_file)
data = metrics[["epoch", "train_acc_epoch", "train_loss_epoch", "val_acc", "val_loss"]]
# Nach Epoche gruppieren, fehlende Werte nach unten füllen
data_cleaned = data.groupby("epoch").first().reset_index()
data_cleaned


In [ ]:
# Plot
plt.figure(figsize=(12, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(
    data_cleaned["epoch"],
    data_cleaned["train_acc_epoch"],
    label="Train Accuracy",
    marker="o",
    linestyle="-",
    linewidth=2,
)
plt.plot(
    data_cleaned["epoch"],
    data_cleaned["val_acc"],
    label="Validation Accuracy",
    marker="o",
    linestyle="-",
    linewidth=2,
)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(
    data_cleaned["epoch"],
    data_cleaned["train_loss_epoch"],
    label="Train Loss",
    marker="o",
    linestyle="-",
    linewidth=2,
)
plt.plot(
    data_cleaned["epoch"],
    data_cleaned["val_loss"],
    label="Validation Loss",
    marker="o",
    linestyle="-",
    linewidth=2,
)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Validation Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


Zur Erinnerung: Der erste naive Ansatz zur Verarbeitung dieser Datenmenge in Kapitel 3 erreichte bei den Testdaten eine Korrektklassifizierungsrate von 88%. 
- Im Vergleich dazu funktioniert das kleine RNN leider nicht besonders gut (nur 85%).
- Das liegt zum einen daran, dass nur die ersten 500 Wörter berücksichtigt werden, nicht die vollständige Sequenz. Das RNN kann also nur auf weniger Informationen zugreifen als das Modell aus Kapitel 3. 
- Zum anderen ist `SimpleRNN` nicht besonders gut für die Verarbeitung langer Sequenzen (wie Text) geeignet.
- Andere Arten von RNNs sind hier deutlich leistungsfähiger.

Sehen wir uns also einige ausgeklügeltere Layer an.

## Vanishing Gradients Problem: LSTM- und GRU-Layer

- Es gibt zwei weitere: `LSTM` und `GRU`. 
- In der Praxis werden Sie eigentlich immer einen dieser beiden verwenden, denn `SimpleRNN` ist im Allgemeinen zu schlicht, um wirklich von Nutzen zu sein. 
- Darüber hinaus gibt es ein ernsthaftes Problem: Theoretisch sollte `SimpleRNN` eigentlich in der Lage sein, zum Zeitpunkt `t` auf Informationen über vor vielen Zeitschritten erfolgte Eingaben zuzugreifen, aber in der Praxis ist es unmöglich, so langfristige Abhängigkeiten zu erlernen. Dafür ist das Problem des **verschwindenden Gradienten (vanishing gradients)**  verantwortlich, ein Effekt, der an das erinnert, was bei nicht rekurrenten NNs (Feedforward-Netze) beobachtbar ist, die viele Layer tief sind:
- Wenn einem NN immer mehr Layer hinzugefügt werden, wird es irgendwann untrainierbar. Die theoretischen Gründe hierfür wurden Anfang der 1990er-Jahre von *Sepp Hochreiter, Jürgen Schmidhuber* [1] und *Yoshua Bengio* [2] untersucht.

**LSTM- und GRU-Layer wurden entwickelt, um dieses Problem zu lösen.**

[1] [Sepp Hochreiter und Jürgen Schmidhuber, *Long Short-Term Memory*, Neural Computation 9,
Nr. 8 (1997)](https://direct.mit.edu/neco/article/9/8/1735/6109/Long-Short-Term-Memory)

[2] [Yoshua Bengio, Patrice Simard und Paolo Frasconi, *Learning Long-Term Dependencies with Gradient Descent Is Difficult*, IEEE Transactions on Neural Networks 5, Nr. 2 (1994)](https://ieeexplore.ieee.org/document/279181)


## LSTM-Layer

Betrachten wir zunächst die LSTM-Layer. 
- Der zugrunde liegende Algorithmus namens [Long Short-Term Memory](https://en.wikipedia.org/wiki/Long_short-term_memory) (LSTM, zu Deutsch etwa »langes Kurzzeitgedächtnis «) wurde 1997 von Hochreiter und Schmidhuber entwickelt und war die Krönung ihrer Forschungsarbeiten über das Problem des verschwindenden Gradienten [2].
- Bei diesem Layer handelt es sich um eine Variante des Ihnen schon bekannten `SimpleRNN`-Layers. 
- *Er fügt eine Möglichkeit hinzu, Informationen über viele Zeitschritte hinweg zu erhalten*. 
- Stellen Sie sich ein Förderband vor, das parallel zu der verarbeitenden Sequenz verläuft. Die in der Sequenz enthaltenen Informationen können bei Bedarf zu jedem beliebigen Zeitpunkt auf das Förderband wechseln, sich zu einem späteren Zeitschritt befördern lassen und dort wieder unbeschadet zurückwechseln. 

**Das ist im Wesentlichen das, was der LSTM-Layer leistet: Informationen für den späteren Gebrauch speichern und so verhindern, dass ältere Signale während der Verarbeitung allmählich verschwinden.**
- Hier ist ein [exzellenter Blog von Christopher Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) zu LSTM

Um das im Detail zu verstehen, betrachten wir eine `SimpleRNN`-Zelle. Da es viele verschiedene Gewichtungsmatrizen gibt, erhalten die Matrizen `W` und `U` als Index den Buchstaben `o` wie output (`Wo` und `Uo`).

<img src="Bilder/SimpleRNN_LSTM.png" width="640"  align="center"/>

Diesem Diagramm *fügen wir nun eine weitere Spur hinzu, auf der Informationen über die Zeitschritte hinweg fliessen können*. 
- Die Werte der verschiedenen Zeitschritte bezeichnen wir als `Ct`, wobei das `c` für carry (befördern) steht.
- Die hier beförderte Information bewirkt in der Zelle Folgendes: 
- Sie wird mit der Eingabe und der rekurrenten Verbindung kombiniert (durch ein Tensorprodukt mit einer
Gewichtungsmatrix, der Addition eines Bias-Vektors und der Anwendung einer Aktivierungsfunktion) und beeinflusst so den Zustand, der an den nächsten Zeitschritt übergeben wird (per Aktivierungsfunktion und Multiplikation). 
- Konzeptuell ist dieser Datenfluss eine Modulierung der nächsten Ausgabe und des nächsten Zustands.

So weit, so gut.

<img src="Bilder/simple_LSTM.png" width="640" align="center"/>
Aus einem `SimpleRNN` wird durch Hinzufügen einer Carry-Spur ein `LSTM`-Layer.

Aber bei der Berechnung des nächsten Carry-Werts wird es raffiniert: Sie erfordert
**drei verschiedene Transformationen in Form einer SimpleRNN-Zelle**:

Allerdings besitzen diese Abbildungen jeweils eigene Gewichtungsmatrizen, die
durch die Indizes `i`, `f` und `k` gekennzeichnet werden. Das mag etwas willkürlich erscheinen, aber warten Sie einen Moment. Damit ergibt sich Folgendes:

Durch Kombination von `i_t`, `f_t` und `k_t` ergibt sich der nachfolgende Carry- Zustand (der nächste Wert von `c_t`).

Dies fügen wir dem Diagramm hinzu, und das war’s.
Eigentlich gar nicht so kompliziert, nur ziemlich komplex.

<img src="Bilder/full_LSTM.png" width="640"  align="center"/>
Aus einem `SimpleRNN` wird durch Hinzufügen einer Carry-Spur ein `LSTM`-Layer.

Wenn Sie die Funktionsweise analysieren möchten, können Sie versuchen zu interpretieren, was
diese Operationen bewirken. 
- Man könnte beispielsweise sagen, dass die Multiplikation von `c_t` und `f_t` eine Möglichkeit darstellt, irrelevante Informationen im Carry Datenfluss zu unterdrücken. 
- `i_t` und `k_t` hingegen liefern Informationen über den jetzigen Zustand und aktualisieren die Carry-Spur.

Letztendlich sind solche **Interpretationen mehr oder weniger bedeutungslos**, denn was diese Operationen tatsächlich
bewirken, hängt von den Gewichtungen ab, durch die sie parametrisiert werden. Und die Gewichtungen werden bei jedem Training anhand aller Zeitschritte neu erlernt, was es unmöglich macht, der einen oder anderen Operation einen bestimmten
Zweck zuzuschreiben. 

Die Spezifizierung einer RNN-Zelle (wie gerade beschrieben) begrenzt den Hypothesenraum – den Raum, in dem Sie beim Trainieren nach einer guten Modellkonfiguration suchen –, sie legt jedoch nicht fest, was eine Zelle eigentlich bewirkt, denn das ist die Aufgabe der Gewichtungen. Ein und dieselbe Zelle kann sehr verschiedene Dinge bewirken, wenn sie unterschiedliche Gewichtungen
besitzt. Die Kombination der Operationen, aus der eine RNN-Zelle besteht, sollte eher als eine Reihe von Beschränkungen Ihrer Suche aufgefasst werden, nicht als ein Design im architektonischen Sinn.

- **Die Spezifizierung einer RNN-Zelle (wie gerade beschrieben) begrenzt den Hypothesenraum** – den Raum, in dem Sie beim Trainieren nach einer guten Modellkonfiguration suchen (inductive Bias).
- Sie legt jedoch nicht fest, was eine Zelle eigentlich bewirkt, denn das ist die Aufgabe der Gewichtungen. Ein und dieselbe Zelle kann sehr verschiedene Dinge bewirken, wenn sie unterschiedliche Gewichtungen besitzt. 
- Die Kombination der Operationen, aus der eine RNN-Zelle besteht, sollte eher als eine Reihe von Beschränkungen Ihrer Suche aufgefasst werden, nicht als ein Design im architektonischen Sinn.

Als Forscher gewinnt man den Eindruck, dass man die Auswahl dieser Beschränkungen, also wie die RNN-Zelle implementiert wird, besser den Optimierungsalgorithmen (wie etwa genetischen Algorithmen oder verstärkenden Lernvorgängen) überlässt, nicht menschlichen Entwicklern. 

- Zukünftig werden wir NNs auf diese Weise entwickeln. 
- **Zusammengefasst**: Sie brauchen die Architektur einer bestimmten LSTM-Zelle nicht zu verstehen; das sollte nicht zu den Aufgaben eines Menschen gehören. Denken Sie einfach nur daran, was eine LSTM-Zelle leisten soll: Sie soll es ermöglichen, ältere Informationen zu einem späteren Zeitpunkt wieder aufzunehmen und so dem Problem des verschwindenden Gradienten entgegenwirken.



## Ein konkretes LSTM-Beispiel in Pytorch


Wenden wir uns wieder praktischen Aufgaben zu und erstellen wir ein Modell mit einem LSTM-Layer, das wir mit den IMDb-Daten trainieren. 
- Das NN ähnelt dem gerade vorgestellten SimpleRNN-Layer.
- Wir geben lediglich die Dimensionalität des LSTM-Layers an; 
- bei allen anderen Argumenten (von denen es eine ganze Menge gibt) belassen wir es bei den Vorgaben.


https://lightning.ai/lightning-ai/studios/train-a-recurrent-neural-network-with-pytorch-lightning?section=featured 

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl


class LSTMClassifier(pl.LightningModule):
    def __init__(self, vocab_size=10000, embedding_dim=32, hidden_size=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=hidden_size, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, 1)
        self.loss_fn = nn.BCELoss()

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, emb_dim)
        _, (h_n, _) = self.lstm(x)  # h_n: (1, batch, hidden)
        out = self.fc(h_n.squeeze(0))  # (batch, 1)
        return torch.sigmoid(out)  # (batch, 1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = self.loss_fn(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze(1)
        loss = self.loss_fn(y_hat, y)
        acc = ((y_hat > 0.5) == y).float().mean()
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=1e-3)


from torch.utils.data import TensorDataset, DataLoader, random_split
from lightning.pytorch.loggers import CSVLogger

# Werte aus deinem vorherigen Code:
# input_train (→ torch.tensor), y_train (→ torch.tensor)
dataset = TensorDataset(X_train, y_train)

# 50% Validation-Split
val_size = len(dataset) // 2
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Modell instanziieren
model = LSTMClassifier(vocab_size=10000)

# CSV-Logger für spätere Plot-Auswertung
csv_logger = CSVLogger("logs", name="lstm_model")

# Trainer
trainer = pl.Trainer(max_epochs=10, accelerator="auto", logger=csv_logger)
trainer.fit(model, train_loader, val_loader)


Dieses Mal erreichen wir bei der Validierung eine Korrektklassifizierungsrate von
bis zu 89%. Gar nicht schlecht und erheblich besser als das SimpleRNN-Modell.
Das liegt vor allem daran, dass **LSTM weniger unter dem Problem des verschwindenden
Gradienten zu leiden hat**. 
- Das Ergebnis ist sogar geringfügig besser als das des vollständig verbundenen Ansatzes in Kapitel 3, obwohl hier weniger Daten verwendet werden, denn die Sequenzen werden nach 500 Zeitschritten abgeschnitten, während in Kapitel 3 vollständige Sequenzen genutzt wurden.

## Zusammenfassung
Sie haben Folgendes gelernt:
- Was RNNs eigentlich sind und wie sie funktionieren
- Was LSTM ist und warum es besser für lange Sequenzen geeignet ist als ein naives RNN
- Wie RNN-Layer zur Verarbeitung sequenzieller Daten verwendet werden

Als Nächstes werden wir einige der erweiterten Features von RNNs betrachten, die Ihnen dabei helfen können, die Deep-Learning-Modelle für sequenzielle Daten richtig auszureizen.